In [1]:
import numpy as np
import pandas as pd
import requests
import io
import re

In [2]:
app_id = '9dd99282'
app_key = '5a7af113a3c5bed96c076726ff56332b'
url = 'https://apis.berkeley.edu/coolclimate/footprint-defaults?input_location_mode=1&input_location=94704&input_income=1&input_size=1'
response = requests.get(url, auth=(app_id, app_key))

In [3]:
response.status_code

200

In [4]:
xml_data = response.content.decode('utf-8')
parsed_data = np.array(re.findall(r'<(\w+)\b>(\d+\.*\d*)<\/\1>', xml_data))

features = parsed_data[:, 0]
numbers = np.array(parsed_data[:, 1], dtype=float)
numbers = numbers.reshape((1, numbers.shape[0]))

df = pd.DataFrame(numbers, columns=features)
df.head()

input_location  input_location_mode  input_size  input_income  \
0         94704.0                  1.0         1.0           1.0   

   input_population  input_changed  input_footprint_household_adults  \
0           21471.0            0.0                               1.0   

   input_footprint_household_children  \
0                                 0.0   

   input_footprint_transportation_num_vehicles  \
0                                          1.0   

   input_footprint_transportation_miles1  ...  \
0                                 5400.0  ...   

   result_takeaction_offset_housing_transpledges  \
0                                            0.0   

   result_takeaction_low_carbon_diet_meatcal  \
0                                     244.35   

   result_takeaction_low_carbon_diet_dairycal  \
0                                       185.9   

   result_takeaction_low_carbon_diet_fvcal  \
0                                    487.8   

   result_takeaction_low_carbon_diet_cerealscal  \
0                                        970.05   

   result_takeaction_low_carbon_diet_othercal  \
0                                       331.2   

   result_takeaction_low_carbon_diet_totalcal  \
0                                      2219.3   

   result_takeaction_offset_shopping_transtonsleft  \
0                                         7.760592   

   result_takeaction_offset_shopping_transpledges  internal_vehiclemiles  
0                                             0.0            5425.048339  

[1 rows x 334 columns]

In [5]:
df.columns

Index(['input_location', 'input_location_mode', 'input_size', 'input_income',
       'input_population', 'input_changed', 'input_footprint_household_adults',
       'input_footprint_household_children',
       'input_footprint_transportation_num_vehicles',
       'input_footprint_transportation_miles1',
       ...
       'result_takeaction_offset_housing_transpledges',
       'result_takeaction_low_carbon_diet_meatcal',
       'result_takeaction_low_carbon_diet_dairycal',
       'result_takeaction_low_carbon_diet_fvcal',
       'result_takeaction_low_carbon_diet_cerealscal',
       'result_takeaction_low_carbon_diet_othercal',
       'result_takeaction_low_carbon_diet_totalcal',
       'result_takeaction_offset_shopping_transtonsleft',
       'result_takeaction_offset_shopping_transpledges',
       'internal_vehiclemiles'],
      dtype='object', length=334)

---

Trying to adjust the default options [`/footprint-default`](https://api-central.berkeley.edu/api/11/interactive-docs#/default/getDefaultFootprint) options

In [7]:
jk_ca = pd.read_csv('../../data/jones-kammen-ca-data.csv')
zip_codes = jk_ca['ZipCode']

In [ ]:
# possible values
input_income = np.arange(1, 12, 1, dtype=int)
input_size = np.arange(1, 6, dtype=int)
input_location_mode = {'zip code' : 1, 'city' : 2, 'county' : 3, 'state' : 4} # granularity is in order: smallest to largest

# customize this!
mode = input_location_mode['zip code']

# initialize empty dataframe
data = pd.DataFrame(columns=df.columns)

for inc in input_income:
    for size in input_size:
        for zc in zip_codes:
            # get the data from the API for this iteration
            url = 'https://apis.berkeley.edu/coolclimate/footprint-defaults?input_location_mode=%d&input_location=%d \
                    &input_income=%d&input_size=%d' % (mode, zc, inc, size)

            response = requests.get(url, auth=(app_id, app_key))

            # parse the requested data from the api
            xml_data = response.content.decode('utf-8')
            parsed_data = np.array(re.findall(r'<(\w+)\b>(\d+\.*\d*)<\/\1>', xml_data))

            # if the parsed data is not empty
            if parsed_data.size > 0:
                features = parsed_data[:, 0]
                observations = np.array(parsed_data[:, 1], dtype=float)
                observations = observations.reshape((1, observations.shape[0]))

                # append to our dataframe
                df = pd.DataFrame(observations, columns=features)
                df['Zip Code'] = zc
                data = data.append(df)

In [ ]:
# change each time you run!
data.to_csv('../../data/coolclimate-api-CAzips-allhouseholds-withzc.csv')

In [ ]:
data.shape

In [118]:
data.columns

Index(['failed', 'error_code', 'input_location_mode', 'input_size',
       'input_income', 'input_population', 'input_changed',
       'input_footprint_household_adults',
       'input_footprint_household_children',
       'input_footprint_transportation_num_vehicles',
       ...
       'result_takeaction_low_carbon_diet_meatcal',
       'result_takeaction_low_carbon_diet_dairycal',
       'result_takeaction_low_carbon_diet_fvcal',
       'result_takeaction_low_carbon_diet_cerealscal',
       'result_takeaction_low_carbon_diet_othercal',
       'result_takeaction_low_carbon_diet_totalcal',
       'result_takeaction_offset_shopping_transtonsleft',
       'result_takeaction_offset_shopping_transpledges',
       'internal_vehiclemiles', 'Zip Code'],
      dtype='object', length=336)

In [119]:
data.head()

failed error_code  input_location_mode  input_size  input_income  \
0    NaN        NaN                  1.0         1.0           1.0   
0    NaN        NaN                  1.0         1.0           1.0   
0    NaN        NaN                  1.0         1.0           1.0   
0    NaN        NaN                  1.0         1.0           1.0   
0    NaN        NaN                  1.0         1.0           1.0   

   input_population  input_changed  input_footprint_household_adults  \
0           54587.0            0.0                               1.0   
0           44342.0            0.0                               1.0   
0           59003.0            0.0                               1.0   
0           67859.0            0.0                               1.0   
0           43032.0            0.0                               1.0   

   input_footprint_household_children  \
0                                 0.0   
0                                 0.0   
0                                 0.0   
0                                 0.0   
0                                 0.0   

   input_footprint_transportation_num_vehicles  ...  \
0                                          1.0  ...   
0                                          1.0  ...   
0                                          1.0  ...   
0                                          1.0  ...   
0                                          1.0  ...   

   result_takeaction_low_carbon_diet_meatcal  \
0                                     244.35   
0                                     244.35   
0                                     244.35   
0                                     244.35   
0                                     244.35   

   result_takeaction_low_carbon_diet_dairycal  \
0                                       185.9   
0                                       185.9   
0                                       185.9   
0                                       185.9   
0                                       185.9   

   result_takeaction_low_carbon_diet_fvcal  \
0                                    487.8   
0                                    487.8   
0                                    487.8   
0                                    487.8   
0                                    487.8   

   result_takeaction_low_carbon_diet_cerealscal  \
0                                        970.05   
0                                        970.05   
0                                        970.05   
0                                        970.05   
0                                        970.05   

   result_takeaction_low_carbon_diet_othercal  \
0                                       331.2   
0                                       331.2   
0                                       331.2   
0                                       331.2   
0                                       331.2   

   result_takeaction_low_carbon_diet_totalcal  \
0                                      2219.3   
0                                      2219.3   
0                                      2219.3   
0                                      2219.3   
0                                      2219.3   

   result_takeaction_offset_shopping_transtonsleft  \
0                                         8.859825   
0                                         8.661071   
0                                         8.578153   
0                                         8.836064   
0                                         8.206639   

   result_takeaction_offset_shopping_transpledges  internal_vehiclemiles  \
0                                             0.0            8669.126912   
0                                             0.0            8490.736995   
0                                             0.0            7986.883857   
0                                             0.0            7308.903795   
0                                             0.0            6013.991938   

   Zip Code  
0   90001.0  
0 

In [120]:
data['input_population'].unique().shape

(996,)

In [1]:
data['result_grand_total']

NameError: name 'data' is not defined